In [1]:
import numpy as np
import pandas as pd

In [43]:
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [4]:
df_all = pd.read_excel("master_2019-2020.xlsx")
site_team_machine = pd.read_excel("Sitegroup_machines.xlsx", sheet_name='Sheet2')

In [36]:
df_all.replace({'SiteGroup': {'Gynae': 'GYNAE', # can skip
                              'gynae': 'GYNAE',
                              'GYNae': 'GYNAE',
                              'Breast':'BREAST',
                              'Sarcoma':'SARCOMA',
                              'Head & Neck':'ENT (HEAD & NECK)',
                              'Lung':'LUNG',
                              'lung':'LUNG',
                              'GI Lower':'GI LOWER',
                              'GI Upper':'GI UPPER',
                              'GU':'GENITOURINARY(GU)',
                              'Skin':'SKIN',
                              'Lymphoma':'LYMPHOMA',
                              'Eye':'EYE',
                              'eye':'EYE',
                              'Endocrine':'ENDOCRINE',
                              'Paediatrics':'PAEDIATRIC',
                              'cns':'CNS',
                              'cNS':'CNS'
                             }}, inplace=True)
df_sites = site_team_machine.set_index('Sitegroup')

In [40]:
# E.g. Daily arrival 2020-01-09
dfins = df_all[df_all['CreatedDate'] == '2020-01-09']
df1 = dfins[['MRN','SiteGroup','Sitegroup_y','Intent','Category_x','SimApptDuration','TltDose','TxFracs','TxApptDuration']]

In [62]:
# check contour and planning time computation
dfins['PlanCompl_TxBook']=dfins['FinalPlanCompleteDate']-dfins['TxApptCreatedDt']
dfins['PlanCompl_TxBook']=dfins['PlanCompl_TxBook'].dt.days

dfins[['MRN','Category_x','TimeBtw_Sim_ContourCompl','Tx-TxBook','PlanCompl_TxBook','WaitTime']]


,MRN,Category_x,TimeBtw_Sim_ContourCompl,Tx-TxBook,PlanCompl_TxBook,WaitTime
221,501606,Urgent 2,0.0,2,4.0,6
249,553742,Standard 3,4.0,19,14.0,19
474,876982,Standard 3,6.0,1,14.0,19
932,2112215,Standard 3,0.0,2,0.0,7
1243,2417776,Standard 3,2.0,4,NaN,18
1982,3193336,Standard 3,5.0,5,3.0,27
2536,3850260,Standard 3,3.0,14,12.0,15
3162,4281566,Standard 3,2.0,18,14.0,18
3224,4313737,Urgent 2,NaN,7,6.0,7
3484,4432565,Urgent 2,0.0,3,3.0,4


In [46]:
dict_site = df_sites.to_dict('index')
df1['Team'] = df1['SiteGroup'].map(lambda x: dict_site[x]['Team'])
df1['CT'] = df1['SiteGroup'].map(lambda x: dict_site[x]['CT'])
df1['linac'] = df1['SiteGroup'].map(lambda x: dict_site[x]['linacs'])

In [48]:
df1.head()

,MRN,SiteGroup,Sitegroup_y,Intent,Category_x,SimApptDuration,TltDose,TxFracs,TxApptDuration,Team,CT,linac
221,501606,CNS,CNS,Curative,Urgent 2,30,6000.0,30.0,30,4,2,12 16 17 14
249,553742,ENT (HEAD & NECK),ENT (HEAD & NECK),Curative,Standard 3,45,7000.0,35.0,30,1,3 4,1 3 4 14
474,876982,GYNAE,GYNAE,Palliative,Standard 3,30,3000.0,10.0,30,3,4,2 9 10 15 14
932,2112215,BREAST,BREAST,Adjuvant,Standard 3,30,4240.0,16.0,25,2,3,5 6 7 8 11 14
1243,2417776,GYNAE,GYNAE,Adjuvant,Standard 3,15,4500.0,25.0,30,3,4,2 9 10 15 14


In [ ]:
# go to compute pretreatment time

In [56]:
# checking
df_all[df_all['CreatedDate']=='2020-01-09'][['MRN','Sitegroup_y','Intent','CTWait','TotalWaitTime']]

,MRN,Sitegroup_y,Intent,CTWait,TotalWaitTime
221,501606,CNS,Curative,1,6
249,553742,ENT (HEAD & NECK),Curative,7,19
474,876982,GYNAE,Palliative,7,19
932,2112215,BREAST,Adjuvant,4,7
1243,2417776,GYNAE,Adjuvant,6,18
1982,3193336,GASTROINTESTINAL(GI),Complex Palliative,6,27
2536,3850260,ENT (HEAD & NECK),Curative,5,15
3162,4281566,ENT (HEAD & NECK),Curative,5,18
3224,4313737,ENT (HEAD & NECK),Palliative,5,7
3484,4432565,ENT (HEAD & NECK),Palliative,1,4
